In [2]:
from helpers.scraper import Scraper
from helpers.csv_helper import get_data_from_csv
from helpers.listing_helper import update_listings

scraper = Scraper('https://www.gumtree.com.au')

# Add login functionality to the scraper
scraper.add_login_functionality('https://www.gumtree.com.au/t-login-form.html', 'span.header__my-gumtree-trigger-text', 'gumtree')
scraper.go_to_page('https://www.gumtree.com.au/web/syi')

# Get data for item type listings from csvs/items.csv
# item_listings = get_data_from_csv('items')
# # Publish all of the items into the facebook marketplace
# update_listings(item_listings, 'item', scraper)

{'domain': '.gumtree.com.au', 'expiry': 1720833061, 'httpOnly': False, 'name': 'aam_uuid', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '28352762311384938714502709350013748961'}
{'domain': '.gumtree.com.au', 'expiry': 1720833061, 'httpOnly': False, 'name': 'aam_tnt', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'aamsegid%3D6797281%2Caamsegid%3D6880889'}
{'domain': 'www.gumtree.com.au', 'expiry': 1720833061, 'httpOnly': False, 'name': '_lr_env_src_ats', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'false'}
{'domain': '.gumtree.com.au', 'expiry': 1752405060, 'httpOnly': False, 'name': 'cto_bundle', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'e0kVBF9jWW80NFJDd3BpdVFjUUdleXVVaEkxWmVTYWpRY3NOcWtvOTRPTW5iMEVJWFQyUUVESiUyQnJZS3IlMkJZU00wUmlOU3hMekNZS3VibjRGV1NzeGF0TzhQakRmYkhMOXdTQkxUZGczVVlnRGVJY3pUMDcxdFR6SVo3Y1VMRldWblU0WEMlMkJobTVuQmRrMTJpVGtpVVNQJTJGTmI2USUzRCUzRA'}
{'domain': '.gumtree.com.au', 'expiry': 1751937058, 'httpOnly': False